# Neural network

In [481]:
import sklearn as sk
import numpy as np
import pandas as pd
import time
from sklearn.cross_validation import KFold
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets
import theano.tensor as th
from scipy import misc
import copy
from numpy.random import uniform
from numpy.random import normal
from sklearn.datasets import load_iris

import matplotlib.pylab as plt
%matplotlib inline

In [486]:

class network:
    def __init__(self, layers_list, learning_rate, alpha, activation_functions, cost_func):
        self.layers_count = len(layers_list)
        self.weight_list = []
        self.learning_rate = learning_rate
        self.layers_list = layers_list
        self.alpha = alpha
        self.cost_func = cost_func
        self.activation_function = activation_functions
        for i in range(1, self.layers_count):
            m = np.asarray(uniform(-1, 1, (self.layers_list[i - 1] + 1, self.layers_list[i])))
            self.weight_list.append(m)
            
    def w_out(self):
        for i in range(len(self.weight_list)):
            print self.weight_list[i]
            print '\n'
    
    def sigmoid(self, x):
        one = np.ones(len(x))
        return one / (one + np.exp(-x * self.alpha))
    
    def der_sigmoid(self, x):
        one = np.ones(len(x))
        return (one - self.sigmoid(x)) * self.sigmoid(x) * self.alpha
    
    def hyp_tg(self, x):
        return (np.exp(self.alpha * x) - np.exp(-self.alpha * x)) / (np.exp(self.alpha * x) + np.exp(-self.alpha * x))
    
    def der_hyp_tg(self, x):
        return self.alpha * (1 - hyp_tg(x) ** 2)
    
    def der_logistic_cost(self, y_true, y_pred):
        return ((1.0 - y_true) / (1.0 - y_pred) - y_true / y_pred)
    
    def logistic_cost(self, y_true, y_pred):
        return -1.0 * (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def square_cost(self, y_true, y_pred):
        return 0.5 * ((y_true - y_pred) ** 2)
    
    def der_square_cost(self, y_true, y_pred):
        return y_pred - y_true
    
    def add_first_layer(self, x_shape):
        self.weight_list.insert(0, np.asarray(uniform(-1, 1, (x_shape, self.layers_list[0]))))
    
    def predict(self, x):
        x = np.asarray(x)
        x = np.insert(x, 0, [-1], axis = 1)
        ans = []
        for s_num, sample in enumerate(x):
            v = sample
            for num, layer in enumerate(self.weight_list):
                v = np.dot(layer.T, v)
                v = self.sigmoid(v)
                if (num != self.layers_count - 1):
                    v = np.insert(v, 0, [-1])
            ans.append(v)
        return ans
    
    def fit(self, x, y):
        x = np.asarray(x)
        y = np.asarray(y)
        x = np.insert(x, 0, -1, axis = 1)
        self.add_first_layer(x.shape[1])
        
        cou_iter = 400
        k = 0
        while (k < cou_iter):
            for s_num, sample in enumerate(x):
                neuron_sum = []
                neuron_out = []
                neuron_out.append(np.asarray(sample, dtype = np.float))
                v = sample
                for num, layer in enumerate(self.weight_list):
                    v = np.dot(layer.T, v)
                    neuron_sum.append(v)
                    v = self.sigmoid(v)
                    if (num != self.layers_count - 1):
                        v = np.insert(v, 0, [-1])
                    neuron_out.append(v)
                    
                num = 0
                der_neuron = []
                for layer in reversed(self.weight_list):
                    if (num == 0):
                        der = (self.der_logistic_cost(y[s_num], neuron_out[-1][0]) * self.der_sigmoid(neuron_sum[-1]))
                        der_neuron.append(der)
                        num += 1
                        #if (num == self.layers_count):
                        #    break
                        #print 'all good'
                        #print np.dot(layer[1😏, der) * self.der_sigmoid(neuron_sum[len(neuron_sum) - num - 1])
                        #return 0
                        #der = (layer[1😏).T * np.tile(der, len(layer) - 1) * self.der_sigmoid(neuron_sum[len(neuron_sum) - num - 1])
                        #der_neuron.insert(0, der[0])
                        #num += 1
                    else:
                        if (num == self.layers_count):
                            break

                    der = np.dot(layer[1, der_neuron[0]) * self.der_sigmoid(neuron_sum[len(neuron_sum) - num - 1])
                        der_neuron.insert(0, der)
                        num += 1
                        
                for layer_num in range(self.layers_count):
                    layer = self.weight_list[self.layers_count - layer_num - 1]
                    result_layer = layer.copy()
                    layer = np.asarray(layer, dtype = np.float)
                    for pos, row in enumerate(layer):
                        row1 = row - self.learning_rate * der_neuron[len(der_neuron) - layer_num - 1] * neuron_out[len(neuron_out) - layer_num - 2][pos]
                        result_layer[pos] = row1
                    self.weight_list[self.layers_count - layer_num - 1] = result_layer
                
            k += 1

In [487]:
from sklearn.utils import shuffle

iris = load_iris()
x = iris.data
y = iris.target

#x, y = shuffle(x, y)


#xtest = x[100:]
#ytest = y[100:]
#x = x[:100]
#y = y[:100]

nt = network([4, 1], 0.05, 1.0, ['sigmoid', 'x'], "square", 'reg')
nt.fit(x, y)
ypred = nt.predict(x)
print ypred, y
#print np.sum((ytest - ypred) ** 2.0) / len(ytest)

[array([ 1.87453742]), array([ 1.861004]), array([ 1.85031868]), array([ 1.85318386]), array([ 1.8713455]), array([ 1.89996495]), array([ 1.85095966]), array([ 1.87337447]), array([ 1.83798309]), array([ 1.86731642]), array([ 1.8926337]), array([ 1.8689746]), array([ 1.85793657]), array([ 1.822932]), array([ 1.89956942]), array([ 1.90857634]), array([ 1.88446515]), array([ 1.87336702]), array([ 1.91174469]), array([ 1.8802971]), array([ 1.89744418]), array([ 1.87816836]), array([ 1.83734165]), array([ 1.88083574]), array([ 1.88088547]), array([ 1.87328083]), array([ 1.87498194]), array([ 1.88263263]), array([ 1.87769327]), array([ 1.86265724]), array([ 1.86588702]), array([ 1.88744924]), array([ 1.88941972]), array([ 1.89731895]), array([ 1.86731642]), array([ 1.85917845]), array([ 1.88694816]), array([ 1.86731642]), array([ 1.83483222]), array([ 1.87753326]), array([ 1.86512157]), array([ 1.83018392]), array([ 1.83699189]), array([ 1.87360188]), array([ 1.89473133]), array([ 1.8554562

# Read Data

In [51]:
im = misc.imread('data/big_alphabet_29x29/mutant-0-0-0.bmp', flatten='grey')
alphabet_size = 25
im_size = 29

In [52]:
x = []
y = []
x_test = []
for letter in range(alphabet_size):
    for i in range(8):
        path = "data/big_alphabet_29x29/mutant-" + str(letter) + "-" + str(i) + "-0.bmp"
        im = misc.imread(path, flatten='grey')
        x.append(im.reshape(im_size * im_size))
        y.append(letter)

In [53]:
for letter in range(alphabet_size):
    path = "data/big_alphabet_29x29/class-" + str(letter) + ".bmp"
    im = misc.imread(path, flatten='grey')
    x_test.append(im.reshape(im_size * im_size))

In [54]:
nt = network([1000, 26], 0.1, 1.0, ['sigmoid', 'sigmoid'], "logistic", 'class')
nt.fit(x, y)
nt.predict(x_test)

[15,
 24,
 24,
 24,
 24,
 15,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 15,
 24,
 24,
 24,
 24,
 24,
 24,
 22,
 24,
 24]